# Noise as target
A keras implementation of the [Unsupervised learning by predicting noise](https://arxiv.org/abs/1704.05310) paper  
  
Here's an analysis of the paper I rather enjoyed. [link](http://www.inference.vc/unsupervised-learning-by-predicting-noise-an-information-maximization-view-2/)  
  
This is an extraction and reformatting of the implementation my team and I made as part of the "[Mozgalo](https://www.estudent.hr/category/natjecanja/mozgalo/)" competition.  
  
I have the [celebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) dataset on hand, and it's a big and publicly available dataset unlike mozgalo so this demo uses it.  

In [11]:
%matplotlib inline

import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.pooling import MaxPooling2D, GlobalAveragePooling2D

from scipy.optimize import linear_sum_assignment

import matplotlib.pyplot as plt

### config

In [13]:
image_shape = (224,224)
batch_size = 128

## Loading the data!

In [15]:
datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')

gen = datagen.flow_from_directory(
    '../data/',
    target_size=image_shape,
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale'
    )

Found 202599 images belonging to 1 classes.


In [19]:
filenames = gen.filenames
n_images = len(filenames)

## Building the model
This is a [darknet](https://pjreddie.com/darknet/imagenet/) model.  
You can use whatever you wish. One of the things I find so nice about this method is that the models are interchangeable wit

In [9]:
def convolutional_block(inp, filters):
    x = Conv2D(filters=filters, kernel_size=3, strides=(1,1), padding='same', use_bias=False)(inp)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = MaxPooling2D(pool_size=(2,2), strides=2)(x)
    return x

c = 1  # we train on b&w images, as the paper suggests, to make the training objective harder 

input_img = Input(shape=(*image_shape, c))

x = convolutional_block(input_img, 16)

x = convolutional_block(x, 32)

x = convolutional_block(x, 64)

x = convolutional_block(x, 128)

x = convolutional_block(x, 256)

x = convolutional_block(x, 512)

x = convolutional_block(x, 1024)

x = Conv2D(filters=1000, kernel_size=1, strides=(1,1), padding='same', use_bias=False)(x)
x = LeakyReLU()(x)
x = GlobalAveragePooling2D()(x)

model = Model(inputs=input_img, outputs=x)
model.compile(loss='mean_squared_error',
              optimizer='adam')

# print(model.summary())

### or if you prefer, load a trained model

In [ ]:
model = load_model('../models/something')

## Helper functions for training

In [ ]:
def sample_spherical(npoints, ndim=n_features):
    '''
    Generates "npoints" number of vectors of size "ndim"
    such that each vectors is a point on an "ndim" dimensional sphere
    that is, so that each vector is of distance 1 from the center
    
    npoints -- number of feature vectors to generate
    ndim -- how many features per vector
    
    returns -- np array of shape (npoints, ndim), dtype=float64
    '''
    vec = np.random.randn(npoints, ndim)
    vec = np.divide(vec, np.expand_dims(np.linalg.norm(vec, axis=1), axis=1))
    return vec